## File 06 - PCA + Linear Regression
##### Group 12:

##### Hannah Schmuckler, mmc4cv

##### Rob Schwartz, res7cd

In this file, we create a small ML pipeline based on the output from File 02 (Feature creation).

We create a linear regression model, but this time use PCA features that were created in file 02. We compare k-values. 

In [1]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression, LinearRegressionModel, LinearRegressionSummary
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col
from pyspark.sql.functions import log
from pyspark.ml.stat import Correlation

import pandas as pd
import numpy as np
import copy

spark = SparkSession.builder \
        .appName("project") \
        .getOrCreate()

sc = spark.sparkContext

CPU times: user 441 ms, sys: 354 ms, total: 795 ms
Wall time: 5.85 s


### Read in dataframes for train and test sets

This data should have been previously generated: we can find it in the `processed_data` folder.

In [2]:
%%time
trainDF = spark.read.parquet("./processed_data/train.parquet")
testDF = spark.read.parquet("./processed_data/test.parquet")
# trainDF.show(5)

CPU times: user 1.08 ms, sys: 2.12 ms, total: 3.2 ms
Wall time: 3.42 s


### Set up Spark ML pipeline training for linear regression

Here we decide which input columns should be used in order to create our training pipeline. To implement this step, we create the function `generatePipeline(inputCols, outputCol)`. Then, we train the pipeline using this function.

In [3]:
%%time


def generatePipeline(inputCols, outputCol):
    # Select input columns for linear regression
    vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="unscaled_features")
    # Scale features
    ss = StandardScaler(inputCol = "unscaled_features", outputCol = "features", withMean = True, withStd = True)
    # Select output column for linear regression
    lr = LinearRegression(featuresCol="features", labelCol=outputCol)

 
    pipeline = Pipeline(stages=[vecAssembler, ss, lr])
    return pipeline


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


### View the model information

Print out the model coefficients and view the pValues, RMSE and R^2. We define the functions `modelInfo(inputCols, pipelineModel)` and `getEvaluationMetrics(pipelineModel,outputCol,testDF)` to report this information.

In [4]:
def modelInfo(inputCols, pipelineModel):
    # Create a zipped list containing the coefficients and the data
    modelCols = copy.deepcopy(inputCols)
    modelCoeffs = list(pipelineModel.stages[-1].coefficients)
    modelCoeffs.insert(0,pipelineModel.stages[-1].intercept)
    modelCols.insert(0,"intercept")
    modelZippedList = list(map(list, zip(modelCols, modelCoeffs)))
    
    # p-values don't work with PCA
    #pvals = pipelineModel.stages[-1].summary.pValues
    
    # Create the pandas DataFrame
    modelDF = pd.DataFrame(modelZippedList, columns = ['Column name', 'Coefficient'])
    # modelDF['pValues'] = pvals
    return modelDF


In [5]:
# Calculate adjusted r2 (https://towardsdatascience.com/machine-learning-linear-regression-using-pyspark-9d5d5c772b42)
# This function will allow us to calculate the adjusted r-square value when we do PCA later. The default r-square function does not take into account k. 
def adj_r2(r2, inputCols, testDF, k = 0):
    n = testDF.count()
    if k == 0:
        p = len(inputCols)
    else: 
        p = len(inputCols) + k - 1
    
    adjusted_r2 = 1-(((1-r2)*(n-1))/(n-p-1))
    return adjusted_r2

In [6]:
def getEvaluationMetrics(pipelineModel,outputCol,testDF,inputCols,k):
    predDF = pipelineModel.transform(testDF)
    predDF.select(outputCol, "prediction").show(10)
    
    print(predDF)
    
    regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol=outputCol,
    metricName="rmse")
    rmse = regressionEvaluator.evaluate(predDF)

    regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol=outputCol,
    metricName="r2")
    r2 = regressionEvaluator.evaluate(predDF)
      
    # Manually calculate Adjusted r2
    adjusted_r2 = adj_r2(r2, inputCols, testDF, k)
    
    return rmse, r2, adjusted_r2


### PCA
First, we'll see how each PCA variant performs on its own. We've created four variations with different k values: 10, 20, 50, and 100. Our PCA represents the categories of purchases that customers made in month one. 

In [7]:
print("** PCA 10 **")
inputCols = ["pca_purchases10"]

# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 10)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 10 **
Model coefficients
       Column name  Coefficient
0        intercept   423.176077
1  pca_purchases10  -858.458237
+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0|18.388107610861937|
|               0.0|24.058767198471003|
|               0.0|17.916930815727596|
|               0.0|338.05151648295737|
|               0.0|254.21827080568016|
|               0.0| 38.56716882862713|
| 312.4200134277344|20.436658406307515|
|  6009.78010559082|254.21827080568016|
|1627.1900024414062|54.577917695824794|
|1393.8800354003906|131.87719543591697|
+------------------+------------------+
only showing top 10 rows

DataFrame[user_id: int, total_spend: double, total_events: bigint, total_sessions: bigint, T_total_spend: double, avg_session_length: double, sd_session_length: double, avg_interactions_per_session: double, sd_interactions_per_session: double, max_interactions_per_session: bigint, purc

In [8]:
print("** PCA 20 **")
inputCols = ["pca_purchases20"]

# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 20)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 20 **
Model coefficients
       Column name  Coefficient
0        intercept   423.176077
1  pca_purchases20  -858.430058
+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0| 7.688522561828734|
|               0.0|15.150733461153948|
|               0.0|  5.44087589598422|
|               0.0|321.68909696662126|
|               0.0|240.34247634881848|
|               0.0| 38.24288531692082|
| 312.4200134277344| 58.00819926364369|
|  6009.78010559082|240.34247634881848|
|1627.1900024414062| 364.3507162822982|
|1393.8800354003906|116.51253004292721|
+------------------+------------------+
only showing top 10 rows

DataFrame[user_id: int, total_spend: double, total_events: bigint, total_sessions: bigint, T_total_spend: double, avg_session_length: double, sd_session_length: double, avg_interactions_per_session: double, sd_interactions_per_session: double, max_interactions_per_session: bigint, purc

In [9]:
print("** PCA 50 **")
inputCols = ["pca_purchases50"]

# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 50)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 50 **
Model coefficients
       Column name  Coefficient
0        intercept   423.176077
1  pca_purchases50  -858.633698
+------------------+-------------------+
|     T_total_spend|         prediction|
+------------------+-------------------+
|               0.0|-0.0860401434403002|
|               0.0| 11.738630944336819|
|               0.0| -9.883890142834957|
|               0.0|  303.5880923566796|
|               0.0| 227.00887631633898|
|               0.0|  259.3156344746661|
| 312.4200134277344|  62.03615349120412|
|  6009.78010559082| 227.00887631633898|
|1627.1900024414062|  345.2859771044529|
|1393.8800354003906|  99.48662692145513|
+------------------+-------------------+
only showing top 10 rows

DataFrame[user_id: int, total_spend: double, total_events: bigint, total_sessions: bigint, T_total_spend: double, avg_session_length: double, sd_session_length: double, avg_interactions_per_session: double, sd_interactions_per_session: double, max_interactions_per_session

In [10]:
print("** PCA 100 **")
inputCols = ["pca_purchases100"]

# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 100)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 100 **
Model coefficients
        Column name  Coefficient
0         intercept   423.176077
1  pca_purchases100  -858.809043
+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0|128.01180992021096|
|               0.0|-1.716576683878884|
|               0.0| 658.0872617356658|
|               0.0|299.23650781864416|
|               0.0|220.59970954832517|
|               0.0|218.19452970661703|
| 312.4200134277344|61.612941328384636|
|  6009.78010559082|220.59970954832517|
|1627.1900024414062|493.07287786353885|
|1393.8800354003906| 92.17758433384711|
+------------------+------------------+
only showing top 10 rows

DataFrame[user_id: int, total_spend: double, total_events: bigint, total_sessions: bigint, T_total_spend: double, avg_session_length: double, sd_session_length: double, avg_interactions_per_session: double, sd_interactions_per_session: double, max_interactions_per_session: bigint, 

#### The adjusted r-square is highest for the PCA with the highest k value, 100. However, all of them are very close - a very small sacrafice in performance helps with a great reduction in dimensionality. 

In [11]:
print("** PCA 10 + all other features **")
inputCols = ["total_spend", "sd_session_length",
             "cart_pct_of_total_events", "avg_purchases_per_session", "cart_events", "purchase_events", 
             "sessions_with_cart", "sessions_with_view", "pct_sessions_end_cart", "total_spend_log", "total_events_log", "pca_purchases10"]

# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 10)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 10 + all other features **
Model coefficients
                  Column name  Coefficient
0                   intercept   423.176077
1                 total_spend   934.878440
2           sd_session_length    39.758379
3    cart_pct_of_total_events   -31.713678
4   avg_purchases_per_session   -55.521223
5                 cart_events    88.682364
6             purchase_events  -128.992321
7          sessions_with_cart   313.724969
8          sessions_with_view   -57.840846
9       pct_sessions_end_cart   -62.206563
10            total_spend_log   -25.774385
11           total_events_log   -70.292791
12            pca_purchases10   -60.755906
+------------------+-------------------+
|     T_total_spend|         prediction|
+------------------+-------------------+
|               0.0| 107.44056659905914|
|               0.0| 104.66517793216622|
|               0.0| 22.727672460071915|
|               0.0|  234.7951884508281|
|               0.0| -54.97627818989281|
|               0

In [12]:
print("** PCA 20 + all other features **")
inputCols = ["total_spend", "sd_session_length",
             "cart_pct_of_total_events", "avg_purchases_per_session", "cart_events", "purchase_events", 
             "sessions_with_cart", "sessions_with_view", "pct_sessions_end_cart", "total_spend_log", "total_events_log", "pca_purchases20"]

# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 20)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 20 + all other features **
Model coefficients
                  Column name  Coefficient
0                   intercept   423.176077
1                 total_spend   940.965193
2           sd_session_length    39.410965
3    cart_pct_of_total_events   -30.850269
4   avg_purchases_per_session   -55.735568
5                 cart_events    87.057600
6             purchase_events  -112.951036
7          sessions_with_cart   311.985577
8          sessions_with_view   -57.942362
9       pct_sessions_end_cart   -61.599202
10            total_spend_log   -26.390316
11           total_events_log   -70.289921
12            pca_purchases20   -45.598685
+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0| 111.7334753694372|
|               0.0|105.55782701314308|
|               0.0|25.761377134790337|
|               0.0|233.00177689306605|
|               0.0|-57.37355443028122|
|               0.0|-27.5

In [13]:
print("** PCA 50 + all other features **")
inputCols = ["total_spend", "sd_session_length",
             "cart_pct_of_total_events", "avg_purchases_per_session", "cart_events", "purchase_events", 
             "sessions_with_cart", "sessions_with_view", "pct_sessions_end_cart", "total_spend_log", "total_events_log", "pca_purchases50"]


# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 50)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 50 + all other features **
Model coefficients
                  Column name  Coefficient
0                   intercept   423.176077
1                 total_spend   947.130259
2           sd_session_length    38.647519
3    cart_pct_of_total_events   -30.838390
4   avg_purchases_per_session   -55.948996
5                 cart_events    85.088053
6             purchase_events  -142.229661
7          sessions_with_cart   309.788626
8          sessions_with_view   -58.738985
9       pct_sessions_end_cart   -60.991472
10            total_spend_log   -25.502282
11           total_events_log   -69.809756
12            pca_purchases50   -67.042072
+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0|107.76882961794149|
|               0.0| 28.79476282887208|
|               0.0|15.617803524783369|
|               0.0| 232.1570810227544|
|               0.0|-58.16794994968734|
|               0.0| 78.1

In [14]:
print("** PCA 100 + all other features **")
inputCols = ["total_spend", "sd_session_length",
             "cart_pct_of_total_events", "avg_purchases_per_session", "cart_events", "purchase_events", 
             "sessions_with_cart", "sessions_with_view", "pct_sessions_end_cart", "total_spend_log", "total_events_log", "pca_purchases100"]


# sd_session_length breaks it, as does sd_interactions_per_session, sd_purchases_per_session
outputCol = "T_total_spend"

pipeline = generatePipeline(inputCols, outputCol)
pipelineModel = pipeline.fit(trainDF)
    
print("Model coefficients")
print(modelInfo(inputCols, pipelineModel))

evaluationMetrics = getEvaluationMetrics(pipelineModel,outputCol,testDF, inputCols, 100)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

** PCA 100 + all other features **
Model coefficients
                  Column name  Coefficient
0                   intercept   423.176077
1                 total_spend   941.666265
2           sd_session_length    36.791566
3    cart_pct_of_total_events   -29.368443
4   avg_purchases_per_session   -56.254885
5                 cart_events    79.113115
6             purchase_events  -251.672941
7          sessions_with_cart   324.707754
8          sessions_with_view   -61.025653
9       pct_sessions_end_cart   -64.102983
10            total_spend_log   -20.138683
11           total_events_log   -68.384100
12           pca_purchases100  -152.186138
+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0|130.17943376323757|
|               0.0|30.057751146559724|
|               0.0| 605.8833443461938|
|               0.0|235.37122692499605|
|               0.0|-61.09414031876884|
|               0.0| 68.

# Champion Model
#### When compared against a linear regression without the PCA features, performance of the models with PCA suffers. This is not just due to the increased number of features increasing the adjusted r-square. The feature we have encoded with PCA must not be a good predictor.  

In [15]:
# pvalues breaks when used with this so taking that out
def modelInfo2(inputCols, pipelineModel):
    # Create a zipped list containing the coefficients and the data
    modelCols = copy.deepcopy(inputCols)
    modelCoeffs = list(pipelineModel.stages[-1].coefficients)
    modelCoeffs.insert(0,pipelineModel.stages[-1].intercept)
    modelCols.insert(0,"intercept")
    modelZippedList = list(map(list, zip(modelCols, modelCoeffs)))
        
    # Create the pandas DataFrame
    modelDF = pd.DataFrame(modelZippedList, columns = ['Column name', 'Coefficient'])
    
    return modelDF

In [16]:
%%time
print("** Champion Model **")
inputCols = ["total_spend", "sd_session_length",
              "avg_purchases_per_session", "cart_events", 
             "sessions_with_cart", "sessions_with_view", "pct_sessions_end_cart", "total_spend_log", "total_events_log", "pca_purchases10"]

outputCol = "T_total_spend"
# Below creation of pipeline is necessary for crossval to run.
vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")

# Select output column for linear regression
lr = LinearRegression(featuresCol="features", labelCol="T_total_spend")

pipeline = Pipeline(stages=[vecAssembler, lr])
    
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0, 0.01, 0.2, 1, 10]) \
    .addGrid(lr.elasticNetParam, [0, 0.25, 0.5, 0.75, 1]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator().setLabelCol("T_total_spend"),
                          numFolds=4)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.setParallelism(8).fit(trainDF)

** Champion Model **
CPU times: user 3.08 s, sys: 601 ms, total: 3.68 s
Wall time: 2min 47s


In [17]:
print("Best model coefficients")
print(modelInfo2(inputCols, cvModel.bestModel))


evaluationMetrics = getEvaluationMetrics(cvModel.bestModel,"T_total_spend",testDF,inputCols, 10)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

print()
print(cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)])
print()
print(cvModel.avgMetrics)

Best model coefficients
                  Column name  Coefficient
0                   intercept   146.439957
1                 total_spend     0.425120
2           sd_session_length     0.000367
3   avg_purchases_per_session   -91.112432
4                 cart_events     5.532628
5          sessions_with_cart    62.295721
6          sessions_with_view    -2.155294
7       pct_sessions_end_cart  -231.925664
8             total_spend_log    -7.004726
9            total_events_log   -38.530261
10            pca_purchases10     0.000000
+------------------+-------------------+
|     T_total_spend|         prediction|
+------------------+-------------------+
|               0.0|  97.14451746519937|
|               0.0|  55.62075138834193|
|               0.0| -3.792607259772865|
|               0.0| 164.23029775358347|
|               0.0| 23.700282810459797|
|               0.0|   59.8026378580403|
| 312.4200134277344|-12.774374240773113|
|  6009.78010559082|  574.1374007991684|
|1627.190

### CV for best PCA ONLY

In [18]:
%%time
print("** PCA Only **")
inputCols = ["pca_purchases50"]

outputCol = "T_total_spend"
# Below creation of pipeline is necessary for crossval to run.
vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")

# Select output column for linear regression
lr = LinearRegression(featuresCol="features", labelCol="T_total_spend")

pipeline = Pipeline(stages=[vecAssembler, lr])
    
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0, 0.01, 0.2, 1, 10]) \
    .addGrid(lr.elasticNetParam, [0, 0.25, 0.5, 0.75, 1]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator().setLabelCol("T_total_spend"),
                          numFolds=4)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.setParallelism(8).fit(trainDF)

** PCA Only **
CPU times: user 2.88 s, sys: 588 ms, total: 3.46 s
Wall time: 2min 8s


In [19]:
print("Best model coefficients")
print(modelInfo2(inputCols, cvModel.bestModel))


evaluationMetrics = getEvaluationMetrics(cvModel.bestModel,"T_total_spend",testDF,inputCols, 50)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

print()
print(cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)])
print()
print(cvModel.avgMetrics)

Best model coefficients
       Column name  Coefficient
0        intercept     2.780719
1  pca_purchases50  -272.143137
+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0|12.596824864880098|
|               0.0| 36.45282469512711|
|               0.0| 4.562739044819039|
|               0.0| 310.4887668677507|
|               0.0|240.20335068705054|
|               0.0|201.51094719992932|
| 312.4200134277344|31.884430243953364|
|  6009.78010559082|240.20335068705054|
|1627.1900024414062| 326.4408097820845|
|1393.8800354003906| 108.0470169569804|
+------------------+------------------+
only showing top 10 rows

DataFrame[user_id: int, total_spend: double, total_events: bigint, total_sessions: bigint, T_total_spend: double, avg_session_length: double, sd_session_length: double, avg_interactions_per_session: double, sd_interactions_per_session: double, max_interactions_per_session: bigint, purchase_pct